In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import gc
from sklearn.preprocessing import MinMaxScaler

from qiskit.circuit.library import z_feature_map, zz_feature_map, pauli_feature_map
from qiskit.circuit.library import unitary_overlap
from qiskit import transpile
from qiskit.quantum_info import Statevector, state_fidelity
from qiskit_aer import AerSimulator

In [2]:
RELEVANT_FEATURES = [
    1,2,3,4,5,6,7,8
]


TARGET = 0

In [3]:
data = pd.read_csv("SUSY.csv.gz", nrows=10_000, header=None)
data = data.dropna(subset=[TARGET, *RELEVANT_FEATURES])

In [4]:
data_features = MinMaxScaler(feature_range=(-np.pi/2, np.pi/2)).fit_transform(data[RELEVANT_FEATURES])
data_target = data[TARGET]

x_train = data_features[:500]
y_train = data_target[:500]
x_test = data_features[-500:]
y_test = data_target[-500:]

In [16]:
np.sum(y_test == 0)

np.int64(273)

In [5]:
from sklearn import svm
from sklearn import metrics

svm_linear = svm.SVC(kernel='rbf')

for i in range(8):
    x_train_real = x_train*2**(-i)
    x_test_real = x_test*2**(-i)

    svm_linear.fit(x_train_real, y_train)
    predictions = svm_linear.predict(x_test_real)

    met_ac = metrics.accuracy_score(y_true=y_test, y_pred=predictions)
    met_pr = metrics.precision_score(y_true=y_test, y_pred=predictions)
    met_rec = metrics.recall_score(y_true=y_test, y_pred=predictions)
    met_f1 = metrics.f1_score(y_true=y_test, y_pred=predictions)

    print(f"{i}\t\t{met_ac:.4f}\t\t{met_pr:.4f}\t\t{met_rec:.4f}\t\t{met_f1:.4f}")


0		0.7180		0.7722		0.5374		0.6338
1		0.7180		0.7722		0.5374		0.6338
2		0.7180		0.7722		0.5374		0.6338
3		0.7180		0.7722		0.5374		0.6338
4		0.7180		0.7722		0.5374		0.6338
5		0.7180		0.7722		0.5374		0.6338
6		0.7180		0.7722		0.5374		0.6338
7		0.7180		0.7722		0.5374		0.6338


svm-rbf
alpha   accuracy    precision   recall      f1
0		0.7180		0.7722		0.5374		0.6338
1		0.7180		0.7722		0.5374		0.6338
2		0.7180		0.7722		0.5374		0.6338
3		0.7180		0.7722		0.5374		0.6338
4		0.7180		0.7722		0.5374		0.6338
5		0.7180		0.7722		0.5374		0.6338
6		0.7180		0.7722		0.5374		0.6338
7		0.7180		0.7722		0.5374		0.6338

In [ ]:
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap
from tqdm.notebook import tqdm
from sklearn import svm
from sklearn import metrics

class QCalculator:
    def __init__(self, fm):
        self.fm = fm
        self.calculated = {}
        self.kernel = None

    def psi(self, x):
        qc = self.fm.assign_parameters(x)
        return Statevector.from_instruction(qc)

    def resolve_psi(self, mat):
        out_cols = []
        for row in tqdm(mat, leave=False):
            key = tuple(row)
            if key not in self.calculated:
                self.calculated[key] = self.psi(row).data
            out_cols.append(self.calculated[key])
        return np.column_stack(out_cols)

    def quantum_kernel(self, A, B):
        PsiA = self.resolve_psi(A)
        PsiB = self.resolve_psi(B)
        G = PsiA.conj().T @ PsiB
        return np.abs(G)**2

    def calculate_kernel(self, x_train):
        self.x_train = x_train
        self.kernel = self.quantum_kernel(self.x_train, self.x_train) 

    def svm(self, c, y_train):
        self.svm_linear = svm.SVC(kernel='precomputed', C=c)
        self.svm_linear.fit(self.kernel, y_train)

    def predict(self, x_test, y_test):
        x_test_matrix = self.quantum_kernel(x_test, self.x_train)
        self.predictions = self.svm_linear.predict(x_test_matrix)
        return self.predictions
                    

In [ ]:
toprints = []
for i in range(8):
    x_train_real = x_train*2**(-i)
    x_test_real = x_test*2**(-i)

    qc = QCalculator(z_feature_map(feature_dimension=8, reps=4))
    qc.calculate_kernel(x_train_real)
    qc.svm(1, y_train)
    predictions = qc.predict(x_test_real, y_test)

    met_ac = metrics.accuracy_score(y_true=y_test, y_pred=predictions)
    met_pr = metrics.precision_score(y_true=y_test, y_pred=predictions)
    met_rec = metrics.recall_score(y_true=y_test, y_pred=predictions)
    met_f1 = metrics.f1_score(y_true=y_test, y_pred=predictions)
    toprints += [f"{i}\t\t{met_ac:.4f}\t\t{met_pr:.4f}\t\t{met_rec:.4f}\t\t{met_f1:.4f}"]

for x in toprints:
    print(x)

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

0		0.6240		0.6027		0.4037		0.4835
1		0.7020		0.6825		0.5917		0.6339
2		0.7120		0.7372		0.5275		0.6150
3		0.6780		0.7436		0.3991		0.5194
4		0.6060		0.8889		0.1101		0.1959
5		0.5640		0.0000		0.0000		0.0000
6		0.5640		0.0000		0.0000		0.0000
7		0.5640		0.0000		0.0000		0.0000


/home/fdoljanin/repo/final-thesis/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [8]:
x_train_real = x_train*2**(-1)
x_test_real = x_test*2**(-1)

qc = QCalculator(z_feature_map(feature_dimension=8, reps=2))
qc.calculate_kernel(x_train_real)
qc.svm(1, y_train)
predictions = qc.predict(x_test_real, y_test)

met_ac = metrics.accuracy_score(y_true=y_test, y_pred=predictions)
met_pr = metrics.precision_score(y_true=y_test, y_pred=predictions)
met_rec = metrics.recall_score(y_true=y_test, y_pred=predictions)
met_f1 = metrics.f1_score(y_true=y_test, y_pred=predictions)
print(f"{met_ac:.4f}\t\t{met_pr:.4f}\t\t{met_rec:.4f}\t\t{met_f1:.4f}")

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

0.7700		0.8294		0.6211		0.7103


svm-zfm-rep1
alpha   accuracy    precision   recall      f1
0		0.6940		0.7176		0.5374		0.6146
1		0.6940		0.7256		0.5242		0.6087
2		0.7220		0.8929		0.4405		0.5900
3		0.6120		0.9459		0.1542		0.2652
4		0.5460		0.5000		0.0044		0.0087
5		0.5460		0.0000		0.0000		0.0000
6		0.5460		0.0000		0.0000		0.0000
7		0.5460		0.0000		0.0000		0.0000

svm-zfm-rep2
0		0.7360		0.7914		0.5683		0.6615
1		0.7700		0.8294		0.6211		0.7103
2		0.7120		0.7943		0.4934		0.6087
3		0.6280		0.8727		0.2115		0.3404
4		0.5460		0.5000		0.0044		0.0087
5		0.5460		0.0000		0.0000		0.0000
6		0.5460		0.0000		0.0000		0.0000
7		0.5460		0.0000		0.0000		0.0000

svm-zfm-rep4
0		0.6860		0.7134		0.5154		0.5985
1		0.7440		0.7647		0.6300		0.6908
2		0.7420		0.7917		0.5859		0.6734
3		0.7120		0.7986		0.4890		0.6066
4		0.6240		0.9333		0.1850		0.3088
5		0.5460		0.5000		0.0044		0.0087
6		0.5460		0.0000		0.0000		0.0000
7		0.5460		0.0000		0.0000		0.0000

In [ ]:
toprints = []
for i in range(8):
    x_train_real = x_train*2**(-i)
    x_test_real = x_test*2**(-i)

    qc = QCalculator(zz_feature_map(feature_dimension=8, reps=4))
    qc.calculate_kernel(x_train_real)
    qc.svm(1, y_train)
    predictions = qc.predict(x_test_real, y_test)

    met_ac = metrics.accuracy_score(y_true=y_test, y_pred=predictions)
    met_pr = metrics.precision_score(y_true=y_test, y_pred=predictions)
    met_rec = metrics.recall_score(y_true=y_test, y_pred=predictions)
    met_f1 = metrics.f1_score(y_true=y_test, y_pred=predictions)
    toprints += [f"{i}\t\t{met_ac:.4f}\t\t{met_pr:.4f}\t\t{met_rec:.4f}\t\t{met_f1:.4f}"]

for x in toprints:
    print(x)

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

0		0.5420		0.4167		0.0220		0.0418
1		0.5720		0.6444		0.1278		0.2132
2		0.5580		0.5484		0.1498		0.2353
3		0.6100		0.5988		0.4273		0.4987
4		0.6620		0.6706		0.5022		0.5743
5		0.7100		0.7412		0.5551		0.6348
6		0.7180		0.7905		0.5154		0.6240
7		0.6300		0.7386		0.2863		0.4127


svm-zzfm-rep1
0		0.5680		0.5797		0.1762		0.2703
1		0.6300		0.6641		0.3744		0.4789
2		0.6160		0.6174		0.4053		0.4894
3		0.6460		0.6562		0.4626		0.5426
4		0.6980		0.7375		0.5198		0.6098
5		0.7260		0.8261		0.5022		0.6247
6		0.6280		0.8868		0.2070		0.3357
7		0.5520		0.8000		0.0176		0.0345

svm-zzfm-rep2
0		0.5640		0.6667		0.0793		0.1417
1		0.5940		0.6500		0.2291		0.3388
2		0.6180		0.6304		0.3833		0.4767
3		0.6280		0.6273		0.4449		0.5206
4		0.6820		0.7125		0.5022		0.5891
5		0.7060		0.7500		0.5286		0.6202
6		0.6540		0.7368		0.3700		0.4927
7		0.6080		0.6598		0.2819		0.3951

svm-zzfm-rep4
0		0.5420		0.4167		0.0220		0.0418
1		0.5720		0.6444		0.1278		0.2132
2		0.5580		0.5484		0.1498		0.2353
3		0.6100		0.5988		0.4273		0.4987
4		0.6620		0.6706		0.5022		0.5743
5		0.7100		0.7412		0.5551		0.6348
6		0.7180		0.7905		0.5154		0.6240
7		0.6300		0.7386		0.2863		0.4127

In [8]:
toprints = []
for i in range(8):
    x_train_real = x_train*2**(-i)
    x_test_real = x_test*2**(-i)

    qc = QCalculator(pauli_feature_map(feature_dimension=8, reps=4, paulis=['X','Y','Z']))
    qc.calculate_kernel(x_train_real)
    qc.svm(1, y_train)
    predictions = qc.predict(x_test_real, y_test)

    met_ac = metrics.accuracy_score(y_true=y_test, y_pred=predictions)
    met_pr = metrics.precision_score(y_true=y_test, y_pred=predictions)
    met_rec = metrics.recall_score(y_true=y_test, y_pred=predictions)
    met_f1 = metrics.f1_score(y_true=y_test, y_pred=predictions)
    toprints += [f"{i}\t\t{met_ac:.4f}\t\t{met_pr:.4f}\t\t{met_rec:.4f}\t\t{met_f1:.4f}"]

for x in toprints:
    print(x)

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

pauli-1
0		0.5680		0.5797		0.1762		0.2703
1		0.6300		0.6641		0.3744		0.4789
2		0.6160		0.6174		0.4053		0.4894
3		0.6460		0.6562		0.4626		0.5426
4		0.6980		0.7375		0.5198		0.6098
5		0.7260		0.8261		0.5022		0.6247
6		0.6280		0.8868		0.2070		0.3357
7		0.5520		0.8000		0.0176		0.0345

pauli-2
0		0.5640		0.6667		0.0793		0.1417
1		0.5940		0.6500		0.2291		0.3388
2		0.6180		0.6304		0.3833		0.4767
3		0.6280		0.6273		0.4449		0.5206
4		0.6820		0.7125		0.5022		0.5891
5		0.7060		0.7500		0.5286		0.6202
6		0.6540		0.7368		0.3700		0.4927
7		0.6080		0.6598		0.2819		0.3951

pauli-4
0		0.5420		0.4167		0.0220		0.0418
1		0.5720		0.6444		0.1278		0.2132
2		0.5580		0.5484		0.1498		0.2353
3		0.6100		0.5988		0.4273		0.4987
4		0.6620		0.6706		0.5022		0.5743
5		0.7100		0.7412		0.5551		0.6348
6		0.7180		0.7905		0.5154		0.6240
7		0.6300		0.7386		0.2863		0.4127